In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import os
import ssl
import codecs
import traceback
import urllib.parse
import pandas as pd
import csv
from bs4 import BeautifulSoup
import sys
import time

In [2]:
# 社内環境からアクセスするためのおまじない
'''
os.environ["HTTP_PROXY"] = "http://${ProxyサーバのIPアドレス}:${ProxyサーバのPort番号}/"
os.environ["HTTPS_PROXY"] = "http://${ProxyサーバのIPアドレス}:${ProxyサーバのPort番号}/"
'''
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
output_dirname = 'asahi_news'
if not os.path.exists(output_dirname):
    os.mkdir(output_dirname)

In [4]:
# ニュース検索（朝日新聞）のURL
#URL = 'https://sitesearch.asahi.com/sitesearch/?Keywords=%E6%97%A5%E6%9C%AC%E9%9B%BB%E6%B0%97&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_search&sort=2&start=0'
#URL = 'https://sitesearch.asahi.com/sitesearch/?Keywords={0:}&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_search&sort=2&start=0'
#URL = 'https://sitesearch.asahi.com/sitesearch/?Keywords={0:}&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_search&sort=2&start={1:}'
URL = 'https://sitesearch.asahi.com/sitesearch/?Keywords={0:}&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_btn{1:}&sort=2&start={2:}'

def get_links(query=None):
    # Selenium用オプション
    op = Options()
    op.add_argument('--disable-gpu')
    op.add_argument('--disable-extensions')
    op.add_argument('--headless')
    op.add_argument('--ignore-certificate-errors')
    op.add_argument('--ignore-ssl-errors')

    # オプション指定して起動
    driver = webdriver.Chrome(options=op)
    # 明示的な待機
    # 一度設定するとfind_element等の処理時に、要素が見つかるまで指定時間の間繰り返し探索するように
    driver.implicitly_wait(10)

    try:
        for page in range(1, sys.maxsize):
            # HTTPリクエストを発行
            print(URL.format(urllib.parse.quote_plus(query, encoding='utf-8'), page, 20*(page-1)))
            driver.get(URL.format(urllib.parse.quote_plus(query, encoding='utf-8'), page, 20*(page-1)))
            time.sleep(5)

            # HTMLドキュメントからBeautifulSoupを初期化
            soup = BeautifulSoup(driver.page_source, "html.parser")
            #soup = BeautifulSoup(html.content, "lxml")
            # 検索結果のニュース件数表示部分を検索し、件数（Int）を取得
            soupCnt = soup.find('p', id='SearchNum').em.text
            #print(soupCnt, int(soupCnt)//20)
            if int(soupCnt) < 20*page:
                # 全て取得済み
                break
            # 検索結果のニュース部分を検索し、全てを取得
            soupNewsQuery = soup.find('ul', id='SiteSearchResult', class_='ListBlock').find_all('li')
            #print(len(soupNewsQuery))

            articles = []
            for idx, soupNews in enumerate(soupNewsQuery):
                article = {
                    'url': soupNews.a.get('href'),
                    'title': soupNews.find('span', class_='SearchResult_Headline').em.span.text,
                    'datetime': soupNews.find('span', class_='Date').text.replace('(', '').replace(')', ''),
                    'summary': soupNews.find('span', class_='read').span.span.text
                }
                #print(article)
                articles.append(article)
                #print('%2d: %s' %(idx + 1, soupNews.find('span', class_='SearchResult_Headline').em.span.text))

            df = pd.DataFrame(articles)
            # DataFrameからCSVファイルを生成
            # encoding='sjis'だとJupyterLab（CSVTable）上で表示不可なことに注意
            #df.to_csv('asahi_news_query_%s.csv' %query, encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
            df.to_csv('asahi_news_query_%s.csv' %query, encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL, mode='a', header=False) # 追記の場合
            time.sleep(5)
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細
        print(traceback.format_exc().rstrip())
        print('=====')
    else:
        print('Exceptionが起きてなければ実行')
    finally:
        print('Exceptionが起きても実行')
        driver.close()

In [5]:
get_links('コロナ 政府 緊急事態宣言 解除 飲食店 酒')

https://sitesearch.asahi.com/sitesearch/?Keywords=%E3%82%B3%E3%83%AD%E3%83%8A+%E6%94%BF%E5%BA%9C+%E7%B7%8A%E6%80%A5%E4%BA%8B%E6%85%8B%E5%AE%A3%E8%A8%80+%E8%A7%A3%E9%99%A4+%E9%A3%B2%E9%A3%9F%E5%BA%97+%E9%85%92&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_btn1&sort=2&start=0
https://sitesearch.asahi.com/sitesearch/?Keywords=%E3%82%B3%E3%83%AD%E3%83%8A+%E6%94%BF%E5%BA%9C+%E7%B7%8A%E6%80%A5%E4%BA%8B%E6%85%8B%E5%AE%A3%E8%A8%80+%E8%A7%A3%E9%99%A4+%E9%A3%B2%E9%A3%9F%E5%BA%97+%E9%85%92&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_btn2&sort=2&start=20
https://sitesearch.asahi.com/sitesearch/?Keywords=%E3%82%B3%E3%83%AD%E3%83%8A+%E6%94%BF%E5%BA%9C+%E7%B7%8A%E6%80%A5%E4%BA%8B%E6%85%8B%E5%AE%A3%E8%A8%80+%E8%A7%A3%E9%99%A4+%E9%A3%B2%E9%A3%9F%E5%BA%97+%E9%85%92&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_btn3&sort=2&start=40
https://sitesearch.asahi.com/sitesearch/?Keywords=%E3%82%B3%E3%83%AD

In [6]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('asahi_news_query_コロナ 政府 緊急事態宣言 解除 飲食店 酒.csv', encoding='utf-8', names=['url', 'title', 'datetime', 'summary'])
df.head()

,url,title,datetime,summary
0,https://www.asahi.com/articles/ASPBT6SGBPBTUTI...,「日常に一歩、近づいてくれれば」飲食店に久々の活気 時短解除,2021/10/25,新型
1,https://www.asahi.com/articles/ASPBC6RJCPBCIIP...,「国はあれこれ対策を立てるけど…」 コロナに振り回されたススキノ,2021/10/12,新型
2,https://www.asahi.com/articles/DA3S15062745.ht...,景況感、宣言解除に望み託す 苦境の外食・航空業界に商機 日銀短観、回復基調でもペース鈍化,2021/10/02,日本銀行が１日に公表した９月の全国企業短期経済観測調査（短観）は、大企業・製造業の業況判断...
3,https://www.asahi.com/articles/ASPB17F05PB1TIP...,4カ月ぶりに店に明かり ハブ酒で乾杯の客も 沖縄・国際通り,2021/10/01,沖縄県で、4カ月に及んだ
4,https://www.asahi.com/articles/ASPB16HS6PB1ULF...,新橋の居酒屋に乾杯の声 景況感「最悪」の飲食業、宣言解除に望み,2021/10/01,日本銀行が1日に公表した9月の全国企業短期経済観測調査（短観）は、大企業・製造業の業況判断...
